<a href="https://colab.research.google.com/github/earltjr/LA_Sushi_restuarants/blob/master/Sushi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from google.colab import userdata
import pandas as pd
from google.colab import files
from google.colab import drive
import time
import math
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The following section is how the YELP data was acquired

In [ ]:
Yelp_API_key=userdata.get('Yelp_API_key')
HEADERS = {'Authorization': f'Bearer {Yelp_API_key}'}
URL = 'https://api.yelp.com/v3/businesses/search'

In [ ]:
# Parameters for the search
params = {
    'term': 'restaurants',
    'location': 'Los Angeles, CA',
    'categories': 'sushi',
    'limit': 50  # Maximum is 50
}
all_sushi_restaurants=[]
offset=0

#Gather all search results until you hit the end the search
while True:
    params['offset'] = offset
    response = requests.get(URL, headers=HEADERS, params=params)

    if response.status_code == 200:
        print(f"Request successful: {response.status_code}\nAdding restaurants") #debug for connection
        sushi_restaurants = response.json()['businesses']

        if not sushi_restaurants:
            print(f"{len(all_sushi_restaurants)} restaurants added") #debug to know how many restuarants added
            break  # End of results

        all_sushi_restaurants.extend(sushi_restaurants)
        offset += len(sushi_restaurants)
    else:
        print(f"Failed to fetch data: {response.status_code}") #debug unsucessful connection check error code
        break  # Check for errors

Request successful: 200
Adding restaurants
Request successful: 200
Adding restaurants
Request successful: 200
Adding restaurants
Request successful: 200
Adding restaurants
Request successful: 200
Adding restaurants
Request successful: 200
Adding restaurants
Request successful: 200
Adding restaurants
Request successful: 200
Adding restaurants
Request successful: 200
Adding restaurants
Request successful: 200
Adding restaurants
420 restaurants added


In [ ]:
df=pd.DataFrame(all_sushi_restaurants)


In [ ]:
filename='yelp_sushi_restaurants.csv'
file_path = f'/content/drive/My Drive/Los Angeles Sushi Restaurant Project/{filename}'
df.to_csv(file_path, index=False)

This section

In [ ]:
Google_Places_API_key=userdata.get('Google_Places_API_key')

In [ ]:
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

search_locations = [
    (34.052235, -118.243683),  # Downtown Los Angeles
    (34.092809, -118.328661),  # Hollywood
    (34.019454, -118.491191),  # Santa Monica
    (34.147785, -118.144516),  # Pasadena
    (33.770050, -118.193739),  # Long Beach
    (34.142507, -118.255075),  # Glendale
    (34.180839, -118.308966),  # Burbank
    (34.073620, -118.400356),  # Beverly Hills
    (34.063145, -118.445516),  # Westwood
    (33.961680, -118.353131),  # Inglewood
    (34.021122, -118.396466),  # Culver City
    (34.139729, -118.391409),  # Studio City
    (33.884736, -118.410908),  # Manhattan Beach
    (33.985047, -118.469483),  # Venice
    (34.086464, -118.270293)   # Silver Lake
]

# List of keywords
keywords = ["sushi", "Japanese", "seafood", "sashimi", "tempura", "Japanese cuisine",
            "teppanyaki", "izakaya", "ramen", "udon", "katsu", "yakitori"]

params = {
    'radius': 100000,                      # Search radius in meters (60 km) coves a span of 120km los angeles county is approximately 112 km from north to south and 80km east to west
    'type': 'restaurant',                 # Type of place
    'key': Google_Places_API_key
}

# Make the GET request

response = requests.get(url, params=params)

all_sushi_restaurants = []
for keyword in keywords:
  for lat, lon in search_locations:
      params = params.copy()
      params['location'] = f'{lat},{lon}'
      params['keyword'] = keyword

      print(f"location being searched lat: {lat}, lon: {lon} for keyword {keyword}")

      while True:
          response = requests.get(url, params=params)
          if response.status_code == 200:
              print(f"Request successful: {response.status_code}\nAdding restaurants")  # Debug for connection
              results = response.json()
              all_sushi_restaurants.extend(results.get('results', []))

              next_page_token = results.get('next_page_token')
              print(f"next page token: {next_page_token}")
              if next_page_token:
                  params['pagetoken'] = next_page_token
                  # Delay required by Google API before using the next_page_token
                  time.sleep(2)
              else:
                  # Remove duplicates (based on place_id)
                  unique_sushi_restaurants = {restaurant['place_id']: restaurant for restaurant in all_sushi_restaurants}
                  unique_sushi_restaurants = list(unique_sushi_restaurants.values())
                  print(f"Total unique sushi restaurants found: {len(unique_sushi_restaurants)}")
                  time.sleep(5)
                  break
          else:
              print(f"Failed to fetch data: {response.status_code}")
              break



location being searched lat: 34.052235, lon: -118.243683 for keyword sushi
Request successful: 200
Adding restaurants
next page token: AUc7tXWyrRVCucHGbYRcPUeN_XpOU91aiNe2tR5oHQDtBqURrVOtfbvBNaKQsHjHC2l9KMKEeRpfq9uiaPFB9BAXqJC14tYDcrJZ6o01zWVyDXrz--ywNRrIZt85ghZ04OaRsS9TAH0xRKzhtdItwuu5CeOFOQ8rMUJCsLib0ZHDVZHtJf8QTVxPP2HRJL_uzmFsu011si2CowLeIteX-hax_XeLj9gmZP-lKbEERrUfvmCq4V9ReXQ0r9sCUzfNQ6JqzdX9i73BzRBYfx2jONReW6K9cgRRdLE_1czs23wCMcqA9lEa_OKK79su-Bf3ynZXeqC2K0kWFj2XnupfGB7sfbL97pBIxNga5WBQnwzaL4aYS2PQ4iDHHGgH0GmKu9hPg1Cc8gouAu6Rkq30GP6tcQ6KCCi8C969PLKNWqHLJMT0XabsIl4-v7S5WV8vWgxaCt-Clw
Request successful: 200
Adding restaurants
next page token: AUc7tXUrhzqkAYL_IDLKjnwTSBH-yIBblFuJqNBAcGz9uXnWOgNbIGPQucbouzGzPr3zKPQFMfYV2tfYZDR91kcjgx9F3GZRNoyPEfNnbipVhS5SCA9Yee89o5Lwrb3lDYO1i_SkZnbid-g7p3RftFJP_4HSI17GdQlcMskyVIvaaj4xdywcVIPMHN7l4YZdrwACNIPyy4FSOd-xxEBzendLYheBh1PPx_wWVAQfpaQPmfbBuz1NGrR5vuOk4sIORw9Tk9uvJrncHhJBFHWiH4lxmroR5cByQSDPrcDHAS02gFYW5yXSNfj50D_mDF-oEZlQYL_piMbL4lMRii-d7by5NH

In [ ]:
df=pd.DataFrame(unique_sushi_restaurants)
filename='google_sushi_restaurants.csv'
file_path = f'/content/drive/My Drive/Los Angeles Sushi Restaurant Project/{filename}'
df.to_csv(file_path, index=False)